# Libraries Used

In [65]:
import vtk
import SimpleITK as sitk
from vtk.util import numpy_support
import numpy as np
import os

# Path Of Folder Carry Project


In [66]:
os.chdir(r"C:\Study\HIS 2\DICOM-Viewer-Features")


# Loading DICOM Files


In [67]:
def load_dicom_series(directory):
    """
    Load a DICOM series from a directory.
    """
    reader = sitk.ImageSeriesReader()
    dicom_files = reader.GetGDCMSeriesFileNames(directory)
    reader.SetFileNames(dicom_files)
    image = reader.Execute()
    return sitk.GetArrayFromImage(image), image.GetSpacing()

# Loading NIfTI Files


In [68]:
def load_nifti_file(file_path):
    """
    Load a NIfTI file.
    """
    image = sitk.ReadImage(file_path)
    return sitk.GetArrayFromImage(image), image.GetSpacing()

# Loading Series Imgs


In [69]:
def load_image_series(directory_or_file):
    """
    Load either a DICOM series or NIfTI file based on input type.
    """
    if os.path.isdir(directory_or_file):  # Directory implies DICOM series
        return load_dicom_series(directory_or_file)
    
    elif directory_or_file.lower().endswith(('*.nii', '*.nii.gz')):  # NIfTI file
        return load_nifti_file(directory_or_file)
    else:
        raise ValueError("Unsupported file type or directory")

# Create dynamic transfer functions


In [70]:
# Global variables to keep track of intensity values
min_intensity = None
max_intensity = None
def update_transfer_functions(color_transfer_function, opacity_transfer_function):
    global min_intensity, max_intensity
    # delete previous points
    color_transfer_function.RemoveAllPoints()
    opacity_transfer_function.RemoveAllPoints()
    # add new points for the updated intensity range
    color_transfer_function.AddRGBPoint(min_intensity, 0.0, 0.0, 0.0)  # Black for low intensity
    color_transfer_function.AddRGBPoint(max_intensity, 1.0, 1.0, 1.0)  # White for high intensity
    # Define opacity function
    opacity_transfer_function.AddPoint(min_intensity, 0.0)  # Fully transparent at min intensity
    opacity_transfer_function.AddPoint(max_intensity, 1.0)  # Fully opaque at max intensity


# Contrast Adjustment


In [71]:
def adjust_contrast(interactor, color_transfer_function, opacity_transfer_function,render_window):   
    global min_intensity, max_intensity
    key = interactor.GetKeySym() #trace the pressed key 
    contrast_step = 0.05  # 5% step size for gradual change
    if key == 'Right':
        # inc contrast (narrow the intensity range)
        min_intensity *= (1 - contrast_step)  # dec the min intensity by 5%
        max_intensity *= (1 + contrast_step)  # inc the max intensity by 5%
        update_transfer_functions(color_transfer_function, opacity_transfer_function)
        render_window.Render()  # Force update of the render window
        print(f"Contrast increased: min={min_intensity}, max={max_intensity}")
    elif key == 'Left':
        # dec contrast (widen the intensity range)
        min_intensity *= (1 + contrast_step)  # inc the min intensity by 5%
        max_intensity *= (1 - contrast_step)  # dec the max intensity by 5%
        update_transfer_functions(color_transfer_function, opacity_transfer_function)
        render_window.Render()  # Force update of the render window
        print(f"Contrast dec: min={min_intensity}, max={max_intensity}")


# Volume Rendering


In [82]:
def create_volume_renderer(volume_data, spacing):
    """
    Create a volume renderer using VTK for the given 3D volume data.
    """
    # Convert the numpy array to a VTK image
    vtk_image = vtk.vtkImageData()
    depth_array = numpy_support.numpy_to_vtk(num_array=volume_data.ravel(), deep=True, array_type=vtk.VTK_FLOAT)
    vtk_image.SetDimensions(volume_data.shape[::-1])
    vtk_image.GetPointData().SetScalars(depth_array)
    vtk_image.SetSpacing(spacing)
    # Create the volume mapper
    volume_mapper = vtk.vtkSmartVolumeMapper()
    volume_mapper.SetInputData(vtk_image)
    # Set the volume property
    color_transfer_function = vtk.vtkColorTransferFunction()
    opacity_transfer_function = vtk.vtkPiecewiseFunction()
    volume_property = vtk.vtkVolumeProperty()
    volume_property.SetColor(color_transfer_function)
    volume_property.SetScalarOpacity(opacity_transfer_function)
    volume_property.ShadeOn()  # Enable shading for realism
    # Create the volume actor
    volume = vtk.vtkVolume()
    volume.SetMapper(volume_mapper)
    volume.SetProperty(volume_property)
    # Create the renderer and add the volume actor
    renderer = vtk.vtkRenderer()
    renderer.AddVolume(volume)
    renderer.SetBackground(0.1, 0.1, 0.1)  # Dark gray background
    renderer.ResetCamera()
    # Set up the render window and interactor
    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)
    render_window.SetSize(800, 600)
    render_interactor = vtk.vtkRenderWindowInteractor()
    render_interactor.SetRenderWindow(render_window)
    # Bind the contrast adjustment functionality
    global min_intensity, max_intensity
    min_intensity = np.min(volume_data)
    max_intensity = np.max(volume_data)
    render_interactor.AddObserver("KeyPressEvent", lambda obj, event: adjust_contrast(obj,color_transfer_function,opacity_transfer_function,render_window))

    return render_window, render_interactor,renderer

## Testing DICOM

In [ ]:
dicom_directory = r"./series-000001"
print("Loading DICOM series...")
volume_data, spacing = load_image_series(dicom_directory)
print("Setting up the volume renderer...")
render_window, render_interactor,_ = create_volume_renderer(volume_data, spacing)
print("Rendering volume...")
render_window.Render()
render_interactor.Start()

Loading DICOM series...
Setting up the volume renderer...
Rendering volume...
Contrast increased: min=-950.0, max=3095.4
Contrast increased: min=-902.5, max=3250.17
Contrast increased: min=-857.375, max=3412.6785000000004
Contrast increased: min=-814.5062499999999, max=3583.3124250000005
Contrast increased: min=-773.7809374999998, max=3762.4780462500007
Contrast increased: min=-735.0918906249998, max=3950.601948562501
Contrast increased: min=-698.3372960937497, max=4148.1320459906265
Contrast increased: min=-663.4204312890622, max=4355.538648290158
Contrast increased: min=-630.2494097246091, max=4573.315580704666
Contrast dec: min=-661.7618802108395, max=4344.649801669432
Contrast dec: min=-694.8499742213816, max=4127.41731158596
Contrast dec: min=-729.5924729324506, max=3921.046446006662
Contrast dec: min=-766.0720965790732, max=3724.9941237063285
Contrast dec: min=-804.3757014080269, max=3538.744417521012
Contrast dec: min=-844.5944864784283, max=3361.8071966449616
Contrast dec: min=

## Testing NIfTI

In [ ]:
nifti_directory = r"./NIFTI/sub-C05_rec-2_T1w.nii.gz"
print("Loading NIfTI series...")
volume_data, spacing = load_nifti_file(nifti_directory)
print("Setting up the volume renderer...")
render_window, render_interactor,_ = create_volume_renderer(volume_data, spacing)
print("Rendering volume...")
render_window.Render()
render_interactor.Start()


Loading NIfTI series...
Setting up the volume renderer...
Rendering volume...
Contrast increased: min=0.0, max=1293.6000000000001
Contrast increased: min=0.0, max=1358.2800000000002
Contrast increased: min=0.0, max=1426.1940000000002
Contrast increased: min=0.0, max=1497.5037000000002


# Comparison Mode

In [ ]:
def create_comparison_mode_renderer(volume_data_1, volume_data_2, spacing_1, spacing_2):
    _, _,renderer_1 = create_volume_renderer(volume_data_1, spacing_1)
    _, _,renderer_2 = create_volume_renderer(volume_data_2, spacing_2)
    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer_1)
    render_window.AddRenderer(renderer_2)
    renderer_1.SetViewport(0.0, 0.0, 0.5, 1.0)  # left side
    renderer_2.SetViewport(0.5, 0.0, 1.0, 1.0)  # right side
    render_window.SetSize(1600, 800)
    render_interactor = vtk.vtkRenderWindowInteractor()
    render_interactor.SetRenderWindow(render_window)

    return render_window, render_interactor

# Testing Comparison Mode

In [ ]:
dicom_directory_1 = r"./series-000001"  # CT 
dicom_directory_2 = r"./series-000002"  # MRI 
# DICOM series
volume_data_1, spacing_1 = load_dicom_series(dicom_directory_1)
volume_data_2, spacing_2 = load_dicom_series(dicom_directory_2)
render_window, render_interactor = create_comparison_mode_renderer(volume_data_1, volume_data_2, spacing_1, spacing_2)
render_window.Render()
render_interactor.Start() 